In [49]:
#@title
import pandas as pd
import requests as req
import xml.etree.ElementTree as ET
import numpy as np

# Eckdaten der Immobilie

Hier die Eckdaten zu der zu erwerbenden Immobilie.

Infos zum ISB-Kredit: https://isb.rlp.de/foerderung/551.html

In [50]:
# Kaufpreis in EUR plus Stellplatz
kaufpreis_wohnung = 246000
kaufpreis_stellplatz = 12000
kaufpreis = kaufpreis_wohnung + kaufpreis_stellplatz

# Wohnflaeche in qm
wohnflaeche = 56

# Kaltmiete plus Stellplatz
# Stand 16.05.23: Koblenz ist Mietstufe 5
kaltmiete = wohnflaeche * 6.40 + 50

# Grunderwerbssteuersatz
grunderwerbssteuersatz = 0.05

# Anteil Grundstück
bodenrichtwert = 1000
anteil_grundstueck = bodenrichtwert / (kaufpreis_wohnung / wohnflaeche)

# Laufzeit in Jahre
laufzeit = 25

# Ist Neubau?
is_neubau = True

# ISB-Kredit: Förderung des Baus von Mietwohnungen, Nr. 551
# Gefördert werden 1900€ / m^2 für Wohnungen für Haushalte mit Einkommen
# über der Einkommensgrenze (§ 13 LWoFG + 60 %), 20 Jahre Bindung
is_isb = True
kredit_isb = (2800 +100 +100)* wohnflaeche
zins_isb = []
for i in range(0, 10 * 12):
  zins_isb.append(0.0)

for i in range(10 * 12, 15 * 12):
  zins_isb.append(0.005)

for i in range(15 * 12, laufzeit * 12):
  zins_isb.append(0.01)

zins_isb_misch = np.mean(zins_isb) # Ausgabe als Mischzins auf 20 Jahre
tilgungszuschuss_isb = 0.45 * kredit_isb

# Bereitstellungskredit zur Überbrückung des ISB-Kredits, da dieser erst bei
zins_bereitstellungskredit = 0.0527

# Eigenkapital (ISB: Mindstens 15% der Gesamtsumme)
eigenkapital = kaufpreis * 0.16

# Kreditzins
zins_bank = 0.041

# Anfängliche Tilgungsrate
tilgung = 0.02

print("Eigenkapital: " + f"{eigenkapital:.0f}" + "€")

Eigenkapital: 41280€


# Abschreibung
Zunächst werden die für die Abschreibung geltenden Werte angegeben. Dafür ist es wichtig, die Anschaffungs- und Herstellungskosten zu kennen. Die Abschreibungswerte beziehen sich in der Regel auf die Anschaffungskosten. Die Ausnahme bildet die Sonder-AfA (Abschreibung für Abnutzung) für den Mietwohnungsbau.

Die Anschaffungskosten setzen sich zusammen aus:
* Kaufpreis (ohne Grundstück)
* Notarkosten
* Maklerprovision  
* Grunderwerbsteuer
* Gebühren des Grundbuchamtes
* Gutachterkosten
* Telefon- und Fahrtkosten
* Gerichtskosten (bei Zwangsversteigerungen)

Die Herstellungskosten setzen sich zusammen aus:
* Kosten für die Baugenehmigung
* Kosten für Architekt und Handwerker
* Baumaterial
* Kosten für Erdarbeiten und Ausschachtungsarbeiten
* Kosten für Installationen von Strom-, Wasser-, Abwasser- und Gasleitungen
* Fahrtkosten zur Baustelle
* Kosten für Umzäunung
* Ablöse- und Abstandzahlungen

Quelle: https://www.buhl.de/steuer/ratgeber/abschreibung-immobilien/

In [51]:
#@title
# Werte ab 01.01.2023

# AfA in Prozent von Neubauten
afa_neubau = 0.03

# AfA in Prozent von Bestandsbauten
afa_bestand_nach_1925 = 0.02
afa_bestand_vor_1925 = 0.025

# Sonder-AfA Mietwohnungsbau
# zusätzlich zur normalen Abschreibung in den ersten vier Jahren
# bis maximal 2000€/qm, bis maximal 3000€/qm Anschaffungs- und Herstellungskosten
afa_mietwohnung = 0.05

# Sonder-AfA Denkmalschutz/Sanierungsgebiet
afa_denkmal_first_8_years = 0.09
afa_denkmal_last_4_years = 0.07

# Annahmen

Um eine Berechnung durchführen zu können, müssen verschiedene Annahmen getroffen werden.

In [52]:
#@title
# zu versteuerndes Einkommen in EUR
einkommen = 100000

# Faustformel für Erhaltungskosten (1% des Kaufpreises pro Jahr) oder 1,30€ pro Quadratmeter
#erhaltungskosten_pro_jahr = 0.01 * kaufpreis
#erhaltungskosten_pro_jahr = 777.36
erhaltungskosten_pro_jahr = 1.5 * wohnflaeche * 12

# Faustformel Grundstückswert (60%) und Gebäudewert (40%)
wert_grundstueck = anteil_grundstueck * kaufpreis
wert_gebaeude = (1 - anteil_grundstueck) * kaufpreis

# Preis-/Kostenentwicklung in % (moderat bei 1% pro Jahr)
anstieg_miete = 1.75
anstieg_nebenkosten = 1
anstieg_immobilienpreise = 0

# Annahmen Kaufnebenkosten

Neben dem eigentlichen Kaufpreis gibt es noch weitere Annahmen bzgl. der Kaufnebenkosten, die beispielsweise vom Bundesland abhängen.

In [53]:
#@title
# Makler vom Kaufpreis (Annahme: 3,57% vom Kaufpreis)
kosten_makler = 0 #round(0.0357 * kaufpreis, 2)

# Notar (Annahme: 1.5% vom Kaufpreis)
kosten_notar = round(0.015 * kaufpreis, 2)

# Eintragung Grundbuch (Annahme: 0.5% vom Kaufpreis)
kosten_grundbuch = round(0.005 * kaufpreis, 2)

# Grunderwerbsteuer (Annahme: 3,5% vom Kaufpreis)
kosten_grunderwerb = round(grunderwerbssteuersatz * kaufpreis, 2)

# Summe Nebenkosten
erwerbsnebenkosten = kosten_grunderwerb + kosten_makler + kosten_notar + kosten_grundbuch

print("Erwerbsnebenkosten: " + str(erwerbsnebenkosten) + "€")
print("-- Grunderwerbsteuer: " + str(kosten_grunderwerb) + "€")
print("-- Makler: " + str(kosten_makler) + "€")
print("-- Notar: " + str(kosten_notar) + "€")
print("-- Grundbuchkosten: " + str(kosten_grundbuch) + "€")

Erwerbsnebenkosten: 18060.0€
-- Grunderwerbsteuer: 12900.0€
-- Makler: 0€
-- Notar: 3870.0€
-- Grundbuchkosten: 1290.0€


# Berechnung Kreditkosten

Die Höhe des Kredits.

In [54]:
#@title
# Summe Investitionskosten
investitionskosten = kaufpreis + erwerbsnebenkosten

# Höhe des Bank-Kredits
kreditsumme = investitionskosten - eigenkapital
kredit_bank = kreditsumme - kredit_isb

# Kosten Bereitstellungskredit
bereitstellungskredit = 80000 * zins_bereitstellungskredit * 2 # Bereitstellungskredit für zwei Jahre

print("Kreditsumme: " + str(kreditsumme) + "€")
print("Kreditsumme Bank: " + str(kredit_bank) + "€")
print("Kreditsumme ISB: " + str(kredit_isb) + "€")
print("Kosten Bereitstellungskredit zur Überbrückung ISB: " + str(bereitstellungskredit) + "€")

Kreditsumme: 234780.0€
Kreditsumme Bank: 66780.0€
Kreditsumme ISB: 168000€
Kosten Bereitstellungskredit zur Überbrückung ISB: 8432.0€


In [55]:
#@title
if is_isb:
  anteil_kredit_isb = kredit_isb / kreditsumme
  anteil_kredit_bank = kredit_bank / kreditsumme
  mischzins = anteil_kredit_isb*(zins_isb_misch) + anteil_kredit_bank*zins_bank
print("Mischzins: " + f"{mischzins * 100:.2f}" + "%")

Mischzins: 1.52%


Nun geht es um die Entwicklung des Kredits. Dieser ist maßgeblich von der Tilgungsrate sowie dem Kreditzins abhängig.

In [56]:
#@title
#Kreditsumme zum testen
#kreditsumme = 308383.5

# Laufzeit in Monaten
laufzeit_monate = laufzeit * 12

# Jährliche Annuität
annuitaet_bank = (zins_bank + tilgung) * kredit_bank
annuitaet_monatlich_bank = round(annuitaet_bank / 12, 2)

#annuitaet_isb = (zins_isb_misch + tilgung) * kredit_isb
annuitaet_isb = 0.02 * kredit_isb
annuitaet_monatlich_isb = round(annuitaet_isb / 12, 2)

# Daten
data = {'Monat':range(laufzeit_monate), 'Annuität Bank':(annuitaet_monatlich_bank), 'Annuität ISB':(annuitaet_monatlich_isb)}
df = pd.DataFrame(data)

# Initiale daten
#restschuld = [kreditsumme]
restschuld_bank = [kredit_bank]
restschuld_isb = [kredit_isb]

# Wie hoch ist der Anteil Zins im ersten Monat?
zinshoehe_bank = [round((zins_bank * kredit_bank) / 12, 2)]
zinshoehe_isb = [round((zins_isb[0] * kredit_isb) / 12, 2)]

# Wie hoch ist der Anteil Tilung ersten Monat?
tilgungshoehe_bank = [round(annuitaet_monatlich_bank - zinshoehe_bank[0], 2)]
tilgungshoehe_isb = [round(annuitaet_monatlich_isb - zinshoehe_isb[0], 2)]

# Daten berechnen
for i in range(1,laufzeit_monate):

  # Die Restschuld ergibt aus der alten Restschuld minus der Tilgung
  tmp_restschuld_bank = round(restschuld_bank[i-1] - tilgungshoehe_bank[i-1], 2)
  restschuld_bank.append(tmp_restschuld_bank)

  tmp_restschuld_isb = round(restschuld_isb[i-1] - tilgungshoehe_isb[i-1], 2)
  restschuld_isb.append(tmp_restschuld_isb)

  # Zinshöhe ist die aktuelle Restschuld pro Monat
  tmp_zinshoehe_bank  = round((zins_bank * restschuld_bank[i]) / 12, 2)
  zinshoehe_bank.append(tmp_zinshoehe_bank)

  tmp_zinshoehe_isb  = round((zins_isb[i] * restschuld_isb[i]) / 12, 2)
  zinshoehe_isb.append(tmp_zinshoehe_isb)

  # Tilgung ergibt sich aus der monatlichen Annuität minus der zu zahlenden Zinsen
  tmp_tilgung_bank = round(annuitaet_monatlich_bank - tmp_zinshoehe_bank, 2)
  tilgungshoehe_bank.append(tmp_tilgung_bank)

  tmp_tilgung_isb = round(annuitaet_monatlich_isb - tmp_zinshoehe_isb, 2)
  tilgungshoehe_isb.append(tmp_tilgung_isb)

# Ergebnisse zum DataFrame hinzufügen
df.insert(len(df.columns), "Restschuld Bank", restschuld_bank)
df.insert(len(df.columns), "Zins Bank", zinshoehe_bank)
df.insert(len(df.columns), "Tilgung Bank", tilgungshoehe_bank)
df.insert(len(df.columns), "Restschuld ISB", restschuld_isb)
df.insert(len(df.columns), "Zins ISB", zinshoehe_isb)
df.insert(len(df.columns), "Tilgung ISB", tilgungshoehe_isb)

#Ausgabe
#@title
# Index bei 1 beginnen lassen
#df.index = range(1, len(df) + 1)
#df.style.format(precision=2, thousands=".", decimal=",")

# Steuerersparnis

Die Steuerersparnis hängt vom zu versteuernden Einkommen, dem Grenzsteuersatz und der Verlust-/Gewinnrechnung ab. Gewinne ergeben sich aus den Mieteinnahmen. Verluste aus den abzusetzenden Kosten der Anschaffung / Herstellung der Immobilie, Nebenkosten, Werbungskosten sowie der aufzuwendenden Zinsen.

Es wird ein Anstieg der Mieten `anstieg_miete` und Anstieg der Nebenkosten `anstieg_nebenkosten` mit berücksichtigt.

Hinweis: Die Nebenkosten werden bei den steuerlichen Entlastungen nicht berücksichtigt, da hier große Unsicherheiten sind, wann die Nebenkosten tatsächlich anfallen.

In [57]:
#@title
# Berechnung der Zinsen pro Jahr
zinsen_pro_jahr_bank = []
for i in range(0,laufzeit):
  tmp_zins_pro_jahr = df['Zins Bank'].iloc[i*12:i*12+12].sum()
  tmp_zins_pro_jahr = round(tmp_zins_pro_jahr, 2)
  zinsen_pro_jahr_bank.append(tmp_zins_pro_jahr)

zinsen_pro_jahr_isb = []
for i in range(0,laufzeit):
  tmp_zins_pro_jahr = df['Zins ISB'].iloc[i*12:i*12+12].sum()
  tmp_zins_pro_jahr = round(tmp_zins_pro_jahr, 2)
  zinsen_pro_jahr_isb.append(tmp_zins_pro_jahr)

# Berechnung der Tilgung pro Jahr
tilgung_pro_jahr_bank = []
for i in range(0,laufzeit):
  tmp_tilgung_pro_jahr = df['Tilgung Bank'].iloc[i*12:i*12+12].sum()
  tmp_tilgung_pro_jahr = round(tmp_tilgung_pro_jahr, 2)
  tilgung_pro_jahr_bank.append(tmp_tilgung_pro_jahr)

tilgung_pro_jahr_isb = []
for i in range(0,laufzeit):
  tmp_tilgung_pro_jahr = df['Tilgung ISB'].iloc[i*12:i*12+12].sum()
  tmp_tilgung_pro_jahr = round(tmp_tilgung_pro_jahr, 2)
  tilgung_pro_jahr_isb.append(tmp_tilgung_pro_jahr)

# Mieteinnahmen pro Jahr inklusive Anstieg
mieteinnahmen_mit_anstieg = [12 * kaltmiete]
for i in range(1,laufzeit):
  # Mietsteigerung mit einberechnen. Anstieg Miete in Prozent
  tmp_mieteinnahmen = mieteinnahmen_mit_anstieg[i-1] * (1 + anstieg_miete / 100)
  mieteinnahmen_mit_anstieg.append(tmp_mieteinnahmen)

# Erhaltungs-/Nebenkosten pro Jahr inklusive Anstieg
erhaltungskosten_mit_anstieg = [erhaltungskosten_pro_jahr]
for i in range(1,laufzeit):
  # Mietsteigerung mit einberechnen. Anstieg Miete in Prozent
  tmp_erhaltungskosten = erhaltungskosten_mit_anstieg[i-1] * (1 + anstieg_nebenkosten / 100)
  erhaltungskosten_mit_anstieg.append(tmp_erhaltungskosten)

# Abschreibung pro Jahr
# Bemessungsgrundlage ist der Kaufpreis mit den Erwerbsnebenkosten, anteilig am Wert des Gebäudes
bemessungsgrundlage = investitionskosten * (1 - anteil_grundstueck)
afa_neubau_pro_jahr = round(afa_neubau * bemessungsgrundlage, 2)
afa_bestand_pro_jahr = round(afa_bestand_nach_1925 * bemessungsgrundlage, 2)

# Steuerersparnis pro Jahr
# setze gültige afa
if is_neubau:
  afa = afa_neubau_pro_jahr
else:
  afa = afa_bestand_pro_jahr

data_steuerersparnis = {"Mieteinnahmen":mieteinnahmen_mit_anstieg, "AfA":afa, \
                        "Zinsen Bank":zinsen_pro_jahr_bank,  "Tilgung Bank": tilgung_pro_jahr_bank, \
                        "Zinsen ISB":zinsen_pro_jahr_isb,  "Tilgung ISB": tilgung_pro_jahr_isb, \
                        "zvEK alt":einkommen}
df_steuerersparnis = pd.DataFrame(data_steuerersparnis)
gewinne = []
for i in range(0,laufzeit):
  tmp_gewinn = mieteinnahmen_mit_anstieg[i] - afa - zinsen_pro_jahr_bank[i] - zinsen_pro_jahr_isb[i]- erhaltungskosten_mit_anstieg[i]
  gewinne.append(round(tmp_gewinn, 2))

df_steuerersparnis.insert(len(df_steuerersparnis.columns) - 1, "Gewinn/Verlust", gewinne)

# zvE => Zu versteuerndes Einkommen in EUR
# gibt die Summe von Lohnsteuer und Soli zurück
def get_lohnsteuer_soli(zvE):
  xml = req.get("http://www.bmf-steuerrechner.de/interface/2023AbJuliVersion1.xhtml?code=ext2023&LZZ=1&STKL=1&RE4=" + str(zvE*100)).text
  root = ET.fromstring(xml)
  lohnsteuer = root.find(".//*[@name='LSTLZZ']").get('value')
  soli = root.find(".//*[@name='SOLZLZZ']").get('value')
  return (float(lohnsteuer) + float(soli)) / 100

# Steuerersparnis berechnen
steuer_alt = get_lohnsteuer_soli(einkommen)
zvEK_neu = []
steuer_neu = []
steuerersparnis = []
monatliche_belastung = []
for i in range(0,laufzeit):
  tmp_zvEK_neu = einkommen + gewinne[i]
  zvEK_neu.append(tmp_zvEK_neu)
  tmp_steuer_neu = get_lohnsteuer_soli(tmp_zvEK_neu)
  steuer_neu.append(tmp_steuer_neu)
  tmp_steuerersparnis = steuer_alt - tmp_steuer_neu
  steuerersparnis.append(tmp_steuerersparnis)
  monatliche_belastung.append( (erhaltungskosten_mit_anstieg[i] + zinsen_pro_jahr_bank[i] + tilgung_pro_jahr_bank[i]  + zinsen_pro_jahr_isb[i] + tilgung_pro_jahr_isb[i] - mieteinnahmen_mit_anstieg[i] - tmp_steuerersparnis) / 12 )

df_steuerersparnis.insert(len(df_steuerersparnis.columns), "Steuer vorher", steuer_alt)
df_steuerersparnis.insert(len(df_steuerersparnis.columns), "zvEK neu", zvEK_neu)
df_steuerersparnis.insert(len(df_steuerersparnis.columns), "Steuer neu", steuer_neu)
df_steuerersparnis.insert(len(df_steuerersparnis.columns), "Steuerersparnis", steuerersparnis)
df_steuerersparnis.insert(len(df_steuerersparnis.columns), "Nebenkosten", erhaltungskosten_mit_anstieg)
df_steuerersparnis.insert(len(df_steuerersparnis.columns), "Monatliche Belastung", monatliche_belastung)

In [58]:
#@title
# Index bei 1 beginnen lassen
df_steuerersparnis.index = range(1, len(df_steuerersparnis) + 1)

df_steuerersparnis.style \
  .format(precision=2, thousands=".", decimal=",")

,Mieteinnahmen,AfA,Zinsen Bank,Tilgung Bank,Zinsen ISB,Tilgung ISB,Gewinn/Verlust,zvEK alt,Steuer vorher,zvEK neu,Steuer neu,Steuerersparnis,Nebenkosten,Monatliche Belastung
1,"4.900,80","6.396,51","2.712,59","1.360,93","0,00","3.360,00","-5.216,30",100.000,"26.903,43","94.783,70","24.451,70","2.451,73","1.008,00","90,75"
2,"4.986,56","6.396,51","2.655,73","1.417,79","0,00","3.360,00","-5.083,76",100.000,"26.903,43","94.916,24","24.514,37","2.389,06","1.018,08","89,66"
3,"5.073,83","6.396,51","2.596,49","1.477,03","0,00","3.360,00","-4.947,43",100.000,"26.903,43","95.052,57","24.578,15","2.325,28","1.028,26","88,56"
4,"5.162,62","6.396,51","2.534,77","1.538,75","0,00","3.360,00","-4.807,20",100.000,"26.903,43","95.192,80","24.644,17","2.259,26","1.038,54","87,52"
5,"5.252,97","6.396,51","2.470,52","1.603,00","0,00","3.360,00","-4.662,99",100.000,"26.903,43","95.337,01","24.712,43","2.191,00","1.048,93","86,54"
6,"5.344,89","6.396,51","2.403,53","1.669,99","0,00","3.360,00","-4.514,56",100.000,"26.903,43","95.485,44","24.781,81","2.121,62","1.059,42","85,54"
7,"5.438,43","6.396,51","2.333,76","1.739,76","0,00","3.360,00","-4.361,85",100.000,"26.903,43","95.638,15","24.853,42","2.050,01","1.070,01","84,59"
8,"5.533,60","6.396,51","2.261,08","1.812,44","0,00","3.360,00","-4.204,70",100.000,"26.903,43","95.795,30","24.927,28","1.976,15","1.080,71","83,71"
9,"5.630,44","6.396,51","2.185,36","1.888,16","0,00","3.360,00","-4.042,95",100.000,"26.903,43","95.957,05","25.003,37","1.900,06","1.091,52","82,88"
10,"5.728,97","6.396,51","2.106,49","1.967,03","0,00","3.360,00","-3.876,46",100.000,"26.903,43","96.123,54","25.081,70","1.821,73","1.102,43","82,10"


In [59]:
#@title
zinsen_sum = df_steuerersparnis['Zinsen Bank'].sum() + df_steuerersparnis['Zinsen ISB'].sum()
print("Summe Zinsen: " + f"{zinsen_sum:.0f}" + "€")

Summe Zinsen: 57953€


In [60]:
#@title
print("Summe Steuerersparnis: " + f"{df_steuerersparnis['Steuerersparnis'].sum():.0f}" + "€")

Summe Steuerersparnis: 44309€


In [61]:
#@title
print("Summe Mieteinnahmen: " + f"{df_steuerersparnis['Mieteinnahmen'].sum():.0f}" + "€")

Summe Mieteinnahmen: 152059€


In [62]:
# @title
print("Restschuld Bank: " + f"{restschuld_bank[-1]:.0f}" + "€")
print("Restschuld ISB: " + f"{restschuld_isb[-1] - tilgungszuschuss_isb:.0f}" + "€")

Restschuld Bank: 9034€
Restschuld ISB: 22768€


# Renditen

Es wird zwischen der Bruttomietrendite und der Nettomietrendite unterschieden.

In [63]:
#@title
# 100 x Jahresnettokaltmiete / Immobilienkaufpreis = Bruttomietrendite in %
bruttomietrendite = (100 * 12 * kaltmiete) / kaufpreis
print("Bruttomietrendite: " + f"{bruttomietrendite:.2f}" + "%")

# Jahresreinertrag / Investitionskosten x 100 = Nettomietrendite in %
jahresreinertrag = 12 * kaltmiete - erhaltungskosten_pro_jahr
nettomietrendite = (jahresreinertrag / investitionskosten) * 100
print("Nettomietrendite: " + f"{nettomietrendite:.2f}" + "%")

Bruttomietrendite: 1.90%
Nettomietrendite: 1.41%


Natürlich können die Renditen genauer berechnet werden. Dabei werden die Ausgaben berücksichtigt, die tatsächlich bezahlt werden. Auf der anderen Seite werden die Einnahmen berücksichtigt, die tatsächlich erziehlt werden.

Auf der Einnahmenseite steht da:
*   Mieteinnahmen
*   Steuerersparnis
*   Zukünftiger Verkaufspreis

Auf der Ausgabenseite steht da:
*   Monatliche Nebenkosten
*   Zinsen Bank- und ISB-Kredit
*   Tilgungsrate Bank- und ISB-Kredit
*   Eingesetztes Eigenkapital
*   Restschuld abzüglich Tilgungszuschuss ISB-Darlehen (40%)
*   Kosten für Kredit zur Zwischenfinanzierung ISB-Darlehen

Zum Schluss muss die Einnamenseite mit der Ausgabenseite gegengerechnet werden. Beim Verkaufspreis wird eine jährliche Preissteigerung eingerechnet. Diese ist in der Variable `anstieg_immobilienpreise` hinterlegt.

In [64]:
#@title
# Zinseszins-Formel
def zinseszins (anfangswert, zins, laufzeit):
  return anfangswert * (1 + zins / 100)**laufzeit

def periodenzins (anfangswert, endwert, laufzeit):
  return ((endwert / anfangswert)**(1/laufzeit) - 1) * 100

# Geschätzter Verkaufspreis, inklusive Anstieg Immobilienpreise
verkaufspreis = zinseszins(kaufpreis, anstieg_immobilienpreise, laufzeit)
print("Geschätzter Verkaufspreis: " + f"{verkaufspreis:.2f}" + "€")

# Restschuld aus beiden Krediten zusammen, ohne Tilgungszuschuss
restschuld = restschuld_bank[-1] + restschuld_isb[-1] - tilgungszuschuss_isb
print("Restschuld: " +  f"{restschuld:.2f}" + "€")

# Ausgaben
tilgung_sum = df_steuerersparnis['Tilgung Bank'].sum() + df_steuerersparnis['Tilgung ISB'].sum()
ausgaben = eigenkapital + df_steuerersparnis['Nebenkosten'].sum() + \
  zinsen_sum + tilgung_sum + restschuld + bereitstellungskredit

# Einnahmen
einnahmen = verkaufspreis + df_steuerersparnis['Mieteinnahmen'].sum() + \
  df_steuerersparnis['Steuerersparnis'].sum()

# Summe Einnahmen - Ausgaben
summe = einnahmen - ausgaben
print("Einnahmen - Ausgaben: " + f"{einnahmen:.2f}" + "€ - " + f"{ausgaben:.2f}" + "€ = " + f"{summe:.2f}" + "€")

# Rendite in Prozent
if (summe <= eigenkapital):
  print("Der Gewinn ist geringer als das eingesetzte Eigenkapital. Kein Gewinn, keine Rendite.")
else:
  rendite = periodenzins(eigenkapital, summe, laufzeit)
  print("Nettomietrendite (exakt): " + f"{rendite:.2f}" + "%")

Geschätzter Verkaufspreis: 258000.00€
Restschuld: 31801.78€
Einnahmen - Ausgaben: 454368.40€ - 295820.93€ = 158547.48€
Nettomietrendite (exakt): 5.53%
